**1.REQUIRED LIBRARY**

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

**2.Download dataset**

In [4]:
def download_mnist_datasets():
  train_data = datasets.MNIST(
      root = "data",
      download = True,
      train = True,
      transform = ToTensor()
  )
  validation_data = datasets.MNIST(
      root = "data",
      download = True,
      train = True,
      transform = ToTensor()
  )
  return train_data,validation_data
  #tranform is important it allow for direct transformation if required


In [5]:
if __name__ == "__main__":
  #download MNIST dataset
  train_data,_ = download_mnist_datasets()
  print("MNIST dataset downloaded")

100%|██████████| 9912422/9912422 [00:00<00:00, 241421685.16it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 10569382.59it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 85022391.83it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 9835069.06it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

MNIST dataset downloaded


**2.2: DATA LODER for train data**

In [6]:
BATCH_SIZE = 128
train_data_loader = DataLoader(train_data,batch_size = BATCH_SIZE )

**3.MODEL CREATION**

Model that create will inherit from the module classes that comes directly from pytorch

In [7]:
class FeedForwardNet(nn.Module):
  def __init__(self, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential(
        nn.Linear(28*28,256),
        nn.ReLU(),
        nn.Linear(256,10)
    )
    self.softmax = nn.Softmax(dim = 1)
  # defining the forward method
  def forward(self,input_data):
    flattened_data = self.flatten(input_data)
    logits = self.dense_layers(flattened_data)
    predictions =self.softmax(logits)

    return predictions

## different layer of this model as attribute
#initial layer will flatten the data
#hidden layer ---> output layer
#apply softmax
# Sequential allow us to pack multiple layers together
#linear is euivalend of a dense layer in keras
# linear has two main argument input features and output features
# images are 28x28
# 256 neurons
# ReLU activation function
# 10 classes 0--->9

**3.1:**

In [9]:
if torch.cuda.is_available():
  device = "cuba"
else:
  device = "cpu"
print(f'using {device} device')

using cpu device


In [10]:
feed_forward_net = FeedForwardNet().to(device)

**4.Implementing Train Model**

In [11]:
def train_one_epoch(model,data_loader,loss_fn,optimiser,device):
  for inputs,targets in data_loader:
    inputs,targets = inputs.to(device),targets.to(device)

    #calculating loss
    predictions = model(inputs)
    loss = loss_fn(predictions,targets)
    #backpropagation loss and update weights
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
  print(f'Loss: {loss.item()}')

In [12]:
def train(model,data_loader,loss_fn,optimiser,device,epoch):
  for i in range(epoch):
    print(f'Epoch{i+1}')
    train_one_epoch(model,data_loader,loss_fn,optimiser,device)
    print(".......................................")
  print("train is doen")

**4.1:train model**

instantiate the loss function + optimiser

In [14]:
loss_fn = nn.CrossEntropyLoss()
LEARNING_RATE= 0.01
optimiser = torch.optim.Adam(feed_forward_net.parameters(),lr=LEARNING_RATE)

In [15]:
EPOCH = 10
train(feed_forward_net,train_data_loader,loss_fn,optimiser,device,EPOCH)

Epoch1
Loss: 1.5087028741836548
.......................................
Epoch2
Loss: 1.5102849006652832
.......................................
Epoch3
Loss: 1.4995170831680298
.......................................
Epoch4
Loss: 1.4715684652328491
.......................................
Epoch5
Loss: 1.4832137823104858
.......................................
Epoch6
Loss: 1.471567153930664
.......................................
Epoch7
Loss: 1.5022248029708862
.......................................
Epoch8
Loss: 1.4817062616348267
.......................................
Epoch9
Loss: 1.4719538688659668
.......................................
Epoch10
Loss: 1.4928556680679321
.......................................
train is doen


**5.SAVE the model**

In [17]:
torch.save(feed_forward_net.state_dict(),"feedforwardnet.pth")
print("Model trainded and saved at the path")

Model trainded and saved at the path
